# Convolutional  Neural Networks and Transfer Learning

Today we are going to explore ConvNets for image classification. And how you might be able to use them for your final projects.

There are an insane number of possible arrangements of pixels. for a 512x512 image there are $256^{786432}$ possible arrangments of RGB colors. Even Google only sees and infitesimal fraction of the possible images. 

![](https://silverspaceship.com/static/shot_1.png)
luckily for us, natural images share certain properties that make the problem much more tractable.

Nearby pixels tend to be similar to eachother, and higher level patterns build on top of each other. pixels form lines, lines form shapes. It is rare for an image to contain large amounts of static.

this tendency for nearby pixels to be related, means we can exploit this with something called convolutional filters

![](img/conv-net2.png)

## Convolution

![](img/3D_Convolution_Animation.gif)

## Max pooling

![](img/Max_pooling.png)

What are the layers learning?

First layer
![](img/filt1.jpg)

Looks like Gabor Filters

Subsequent Layers get harder to visualize. ![But Inception networks can give it a go](http://yosinski.com/static/proj/deepvis_all_layers.jpg)

In [1]:
# First few layers of a CNN is a FEATURE EXTRACTOR!!

Famous Pre-trained Image Models
* VGG16
* VGG19
* InceptionV3


## VGG16

![](img/vgg16_croped.png)

Download VGG-16 trained weights from [Here](https://github.com/fchollet/deep-learning-models/releases)

You will want the [tensorflow h5 file](https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels.h5). Note this file is a little over half a gigabyte, so it will take a while to download.

Once you have downloaded it, change `weight_file` below to where you saved the file

In [ ]:
import os

weight_file = '/Users/damien/Downloads/vgg16_weights_tf_dim_ordering_tf_kernels.h5'

if not os.path.exists(weight_file):
    raise FileNotFoundError("No file {weight_file} found. Check path again".format(weight_file))

#### Download labels too

In [ ]:
!curl https://raw.githubusercontent.com/torch/tutorials/master/7_imagenet_classification/synset_words.txt -o synset_words.txt

## A Convolutional Neural Network Architecture in Practice

### VGG (Loading a Pretrained Network, first the harder & less stable way)

See section below for the better way to load a pretrained network using keras utilities.

The hard way is to explicitly define the network structure, and then go find and manually load the learned weights that correspond to that structure. Someone has already trained this network on a large dataset (imagenet), has saved those weights, and has made them publicly downloadable.

**For your image projects, you would very likely want to use a pretrained network as the basis for an image classifier**. You might use something more recent than VGG like ResNet or Google Inception. You would use transfer learning on top of this, as detailed below.

In [ ]:
from keras import backend as K
K.set_image_dim_ordering('th')

from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
import numpy as np
import pandas as pd
import PIL

In [ ]:
def VGG_16(weights_path=None):
    # This is the structure of the neural network
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    # this part is no longer CNN
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))  # randomly drops out half the nodes
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))  # randomly drops out half the nodes. serves to regularize model
    model.add(Dense(1000, activation='softmax'))

    if weights_path:
        model.load_weights(weights_path)

    return model

In [ ]:
synset = pd.read_csv('synset_words.txt', skipinitialspace=True, names = ['synset', 'words'])

# note that we don't actually train/adjust the weights at all here
model = VGG_16(weight_file)  # this part is to load the weights into the model
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy')

In [ ]:
def convert_image_to_bgr_numpy_array(image_path, size=(224,224)):
    """The network has been trained using opencv and BGR images 
    (i.e. channels order blue, green, red rather than red, green, blue).
    The description of why is https://stackoverflow.com/questions/14556545/why-opencv-using-bgr-colour-space-instead-of-rgb
    
    We can use a simpler image library as long as we manually convert
    the data to the expected format.
    """
    image = PIL.Image.open(image_path).resize(size)
    img_data = np.array(image.getdata(), np.float32).reshape(*size, -1)
    # swap R and B channels
    img_data = np.flip(img_data, axis=2)
    return img_data

def prepare_image(image_path):
    im = convert_image_to_bgr_numpy_array(image_path)

    # these subtractions are just mean centering the images 
    # based on known means for different color channels
    im[:,:,0] -= 103.939
    im[:,:,1] -= 116.779
    im[:,:,2] -= 123.68
    
    im = im.transpose((2,0,1)) # adjust from (224, 224, 3) to (3, 224, 224) for keras (for keras you have to tell them aobut how you put in the data)
    im = np.expand_dims(im, axis=0) # adjust to (1, 3, 224, 224) for generating keras prediction
    return im

![](img/dog.jpg)

One way to do this - we're using an explicitly defined model structure and loaded pre-trained weights into it. We can then run the pre-trained model any any image we want, and see which label it predicts. Of course, the predicted label is drawn from the set of labels that the model was trained on (ImageNet).

In [ ]:
img = prepare_image('img/dog.jpg')  # this line is to modify image into right size and format
# the output of this will give you a matrix

out = model.predict(img) 
# this gives you a lot of numbers


y_pred = np.argmax(out) 
# out of all the numbers above, there is going to be the one that is the largest -- and that is the prediction

print(y_pred)
print(synset.loc[y_pred].synset) # synset is the ground truth. the label that was used to train the model



In [ ]:
# synset   
# # if you type out this code, you will see all the images and the 
# # categories of which the model is trained on

## Instead, the preferred and stable way to do this!!

A more stable way to do this - using built in keras applications to load the network we want and tell it to download / use pretrained weights. **This is highly recommended over manually defining architectures and loading weights!**

Here are [more examples of keras transfer learning](https://keras.io/applications/) with modern pretrained CNNs. Check out the documentation specific to the model(s) you want to use.

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.applications.imagenet_utils import decode_predictions

#instead of doing evertyhing ourselves, just import VGG16

# prepare the image yourself
img = prepare_image('img/dog.jpg')

# create a model with the VGG16
model = VGG16(weights='imagenet')

# then do a prediction 
out = model.predict(img)
y_pred = np.argmax(out)


# decode_predictions is gives you other predictions in descending order
print('Predicted:', decode_predictions(out))

Close enough

![](img/sloth.jpg)

In [ ]:
img = prepare_image('img/sloth.jpg')
out = model.predict(img)


print('Predicted:', decode_predictions(out))
# this preidciton actually predicted this as a cup as opposed to a sloth.
# This is probably because in the 1000 images that was trained it wasn't trained on a sloth!


It took a huge amount of gpu time/power and data to train this model. More that you will have access to over the next 3 weeks. So what is one to do if you want to do an image related project?

In [ ]:
# What we did above was transfer learning! We didn't do any training by ourselves
## We can use transfer learning to just pull out images that we want!! (Transfer learning)

**Cheat!**

### Transfer Learning

it turns out that the lower level featured learned by VGG16 on imagenet are still applicable to other problems with natural images. If we can preserve the lower-level features, we can just train a new model on those features. (In fact, in the case of 'softmax', we can think of this as just training a new multinomial logistic regression, on those convolution features)

Lets just snip off last layer.

In [ ]:
# All we need to do is to snip off the last layer and apply it to our own!!

A Caveat

if we just add a new layer with default weights, it is going to be very wrong the first iteration. Since it is so wrong, the gradient will be huge, and because we are using back propagation those errors will be sent down stream into the lower level features. This can quickly destroy the rest of the network.

In order to retrain this model we must protect the lower-level features, until our new layers have reached more stability. We can do this by freezing those layers

Then we'll add our new layer.

In [ ]:
from keras.models import Model

# note we exclude the final dense layers and add one back below, 
# we would retrain it ourselves
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(3,224,224)) 
 
# Freeze convolutional layers  (WE DON'T WANT TO RETRAIN THE ENTIRE MODEL!!)
for layer in base_model.layers:
    layer.trainable = False    
    
# Think of this part as the feature extraction
# Then when we want to do our own customized prediction (maybe.. corgi or not corgi)
## Then we add our own 2 layers, and freeze the feature extractors
### Then we do training on our own last 2 layers with our own images of corgis
    
    
x = base_model.output
# We are adding the following 2 layers to the output
x = Flatten()(x) # flatten from convolution tensor output (remove the last layer) 
predictions = Dense(2, activation='softmax')(x) # should match # of classes predicted

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9),
            loss='categorical_crossentropy', metrics=['accuracy'])

Then you would just train like normal

```python
# i.e. if we had training images and our own labels, we could run
model.fit(X_train,y_train)
```

How much data do you need?

More!

Actually with this bottleneck approach, you don't need as much. 200-1000 representitive images of each class will give good results. Because
* Google has already done most of the hard work
* We can use image augmentation to increase our number of training samples

### Image Augmentation

![](img/DataAugmentation.png)

Possible Augmentations:
* Scale
* Rotation
* Skew
* Flips
* color tinting
* blur
* crops
* ETC

As long as you do not destroy the info you are trying to represent.

[Check out this](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html) for more with keras!

### Other Deep Learning Architectures

#### Auto-Encoders
![](img/Autoencoder.png)

Trying to learn an approximation of our input data after compression step. Autoencoders are super similar to Matrix Factorization, and are completely unsupervised

#### Variational Auto-Encoder
![](http://kvfrans.com/content/images/2016/08/vae.jpg)
If you build in an understanding of noise, you have a Variational Autoencoder (VAE)

#### Siamese Network
![](img/siamese.png)

This tends to be used to compare the inputs. 

- Answer and question matching
- standardization
- facial recognition
- picture captioning

#### Generative Adversarial Networks
![](img/GAN.jpg)
Create a Generative model, and a Discrimative model. Pit them against eachother to improve both

[Really cool example of Adversarial Networks in action](http://carpedm20.github.io/faces/)

#### Stack GANs
Gans can be stacked with awesome results
![](http://i.imgur.com/SGzE7vI.jpg)

https://arxiv.org/pdf/1612.03242v1.pdf

#### image analogy

![](https://raw.githubusercontent.com/awentzonline/image-analogies/master/examples/images/sugarskull-analogy.jpg)
![](https://raw.githubusercontent.com/awentzonline/image-analogies/master/examples/images/image-analogy-explanation.jpg)
![](https://raw.githubusercontent.com/awentzonline/image-analogies/master/examples/images/trump-image-analogy.jpg)

https://github.com/awentzonline/image-analogies

New Architectures are being published every day. So much to read!

* [Curated List of Deep Learning papers](https://github.com/ChristosChristofidis/awesome-deep-learning)
* [Good reddit post for keeping up with the latest research](https://www.reddit.com/r/MachineLearning/comments/6d7nb1/d_machine_learning_wayr_what_are_you_reading_week/)
